# Continuum Mechanics Package Tutorials

In [ ]:
from IPython.display import Image, display

## Beam

Beams are characterized by their length, constraints, cross-sectional second moment of area, and elastic modulus. 
In SymPy, 2D beam objects are constructed by specifying the following properties:

- Length
- Elastic Modulus
- Second Moment of Area
- Variable: A symbol representing the location along the beam’s length. By default, this is set to Symbol(x).
- Boundary Conditions
    - bc_slope: Boundary conditions for slope.
    - bc_deflection: Boundary conditions for deflection.
- Load Distribution

Once the above are specified, the following methods are used to compute useful information about the loaded beam:

- `solve_for_reaction_loads()`
- `shear_force()`
- `bending_moment()`
- `slope()`

## Example

A cantilever beam 9 meters in length has a distributed constant load of 8 kN/m applied downward from the fixed end over a 5 meter distance. A counterclockwise moment of 50 kN-m is applied 5 meters from the fixed end. Lastly, a downward point load of 12 kN is applied at the free end of the beam.

In [ ]:
display(Image(filename="images/ContMechanicsExample.png"))

In [ ]:
from sympy import *
x, y, z = symbols('x y z')

The beam must be initialized with the length, modulus of elasticity, and the second moment of area. These quantities can be symbols or numbers.

In [ ]:
from sympy.physics.continuum_mechanics.beam import Beam
E, I = symbols('E, I')
b = Beam(9, E, I)

The three loads are applied to the beam using the `apply_load()` method. This method supports point forces, point moments, and polynomial distributed loads of any order, i.e. $c, cx, cx^2, cx^3, ...$

The 12 kN point load is in the negative direction, at the location of 9 meters, and the polynomial order is specified as -1:

In [ ]:
b.apply_load(12, 9, -1)

The `load` attribute can then be used to access the loading function in singularity function form:

In [ ]:
b.load

Similarly, the positive moment can be applied with a polynomial order -2:

In [ ]:
b.apply_load(50, 5, -2)

The distributed load is of order 0 and spans x=0 to x=5:

In [ ]:
b.apply_load(8, 0, 0, end=5)

The fixed end imposes two boundary conditions: 

1) no vertical deflection 
2) no rotation. 

These are specified by appending tuples of x values and the corresponding deflection or slope values:

In [ ]:
b.bc_deflection.append((0, 0))
b.bc_slope.append((0, 0))

These boundary conditions introduce an unknown reaction force and moment which need to be applied to the beam to maintain static equilibrium:

In [ ]:
R, M = symbols('R, M')
b.apply_load(R, 0, -1)
b.apply_load(M, 0, -2)
b.load

These two variables can be solved for in terms of the applied loads and the final loading can be displayed:

In [ ]:
b.solve_for_reaction_loads(R, M)
b.reaction_loads

At this point, the beam is fully defined and the internal shear and bending moments are calculated:

In [ ]:
b.shear_force()

In [ ]:
b.bending_moment()

These can be visualized by calling the respective plot methods:

In [ ]:
b.plot_shear_force()
b.plot_bending_moment()

The beam will deform under load and the slope and deflection can be determined with:

In [ ]:
b.slope()

In [ ]:
b.deflection()

In [ ]:
b.plot_slope(subs={E: 20E9, I: 3.25E-6})
b.plot_deflection(subs={E: 20E9, I: 3.25E-6})

All of the plots can be shown in one figure with:

In [ ]:
b.plot_loading_results(subs={E: 20E9, I: 3.25E-6})

## Try yourself

There is a beam of length 30 meters. A moment of magnitude 120 Nm is applied in the counter-clockwise direction at the end of the beam. A point load of magnitude 8 N is applied from the top of the beam at the starting point. There are two simple supports below the beam. One at the end and another one at a distance of 10 meters from the start. The deflection is restricted at both the supports.

In [ ]:
display(Image(filename="images/ContMechanicsTryYourself.png"))

**Solution:**

In [ ]:
from sympy.physics.continuum_mechanics.beam import Beam
from sympy import symbols

E, I = symbols('E, I')
b = Beam(30, E, I)

b.apply_load(8, 0, -1)
b.apply_load(120, 30, -2)

R1, R2 = symbols('R1, R2')
b.apply_load(R1, 10, -1)
b.apply_load(R2, 30, -1)

b.bc_deflection.append((10, 0))
b.bc_deflection.append((30, 0))

b.solve_for_reaction_loads(R1, R2)
b.reaction_loads

b.shear_force()
b.bending_moment()
b.slope()
b.deflection()